In [0]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import gym

In [0]:
env = gym.make('CartPole-v0')

num_episodes = 1000
learning_rate = 0.8
discount = 0.8
input_size = env.observation_space.shape[0]
output_size = env.action_space.n

In [3]:
X = tf.placeholder(shape=[None, input_size], dtype=tf.float32, name='input_x')
Y = tf.placeholder(shape=[None, output_size], dtype=tf.float32)
W1 = tf.get_variable('W1', shape=[input_size, output_size],
                      initializer = tf.keras.initializers.he_normal())
Qpred = tf.matmul(X, W1)
cost = tf.reduce_sum(tf.square(Y - Qpred))
train = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=1e-4).minimize(cost)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
rList = []

sess = tf.Session()
sess.run(tf.global_variables_initializer())
  
for step in range(num_episodes):
  step_cnt = 0
  state = env.reset()
  done = False
  rAll = 0
  e = 1.0 / ((step + 50) + 5)

  while not done:
    step_cnt += 1
    # preprocess
    now_x = np.reshape(state, [-1, input_size])
    # Choose an action by greedily (with e chance of random action) from the Q-net
    Qs = sess.run(Qpred, feed_dict={X: now_x})
    if np.random.rand(1) < e:
      action = env.action_space.sample()
    else:
      action = np.argmax(Qs + np.random.randn(1, output_size) / (step + 1))

    new_state, reward, done, _ = env.step(action)
    if done:
      Qs[0, action] = -100
    else:
      new_x = np.reshape(new_state, [1, input_size])
      Qs1 = sess.run(Qpred, feed_dict={X:new_x})
      Qs[0, action] = reward + discount * np.max(Qs1)

    sess.run(train, feed_dict={X:now_x, Y:Qs})
    state = new_state
  rList.append(step_cnt)
  print("Episode: {}   steps: {}".format(step, step_cnt))
  if len(rList) > 10 and np.mean(rList[-10:]) > 500:
    break

Episode: 0   steps: 42
Episode: 1   steps: 10
Episode: 2   steps: 9
Episode: 3   steps: 200
Episode: 4   steps: 200
Episode: 5   steps: 156
Episode: 6   steps: 29
Episode: 7   steps: 75
Episode: 8   steps: 56
Episode: 9   steps: 77
Episode: 10   steps: 158
Episode: 11   steps: 39
Episode: 12   steps: 38
Episode: 13   steps: 30
Episode: 14   steps: 66
Episode: 15   steps: 31
Episode: 16   steps: 23
Episode: 17   steps: 51
Episode: 18   steps: 32
Episode: 19   steps: 23
Episode: 20   steps: 26
Episode: 21   steps: 22
Episode: 22   steps: 34
Episode: 23   steps: 24
Episode: 24   steps: 21
Episode: 25   steps: 35
Episode: 26   steps: 60
Episode: 27   steps: 75
Episode: 28   steps: 32
Episode: 29   steps: 24
Episode: 30   steps: 30
Episode: 31   steps: 20
Episode: 32   steps: 28
Episode: 33   steps: 22
Episode: 34   steps: 10
Episode: 35   steps: 21
Episode: 36   steps: 76
Episode: 37   steps: 47
Episode: 38   steps: 42
Episode: 39   steps: 29
Episode: 40   steps: 37
Episode: 41   steps: 43

In [5]:
print(Qpred.shape)

(?, 2)


In [6]:
# Test
observation = env.reset()
reward_sum = 0

while True:
  #   env.render()

    x = np.reshape(observation, [1, input_size])
    Qs = sess.run(Qpred, feed_dict={X: x})
    action = np.argmax(Qs)

    observation, reward, done, _ = env.step(action)
    reward_sum += reward

    if done:
      print("Total score: {}".format(reward_sum))
      break
      
sess.close()

Total score: 31.0
